In [ ]:
from dotenv import load_dotenv,find_dotenv
import os
import pandas as pd
from binance.client import Client
import requests
import json
import time
import functions as f 

In [ ]:
# Define the path to the .env file
load_dotenv(find_dotenv('secrets.env'))
api_key = os.getenv('API_KEY')
api_secret = os.getenv('API_SECRET')

# Initialize our client
client = Client(api_key, api_secret, testnet=True)

In [ ]:
# Define our weight variable to control API limits
current_API_weight = 0
MAX_API_WEIGHT = 5500

In [2]:
# Helper function to control API limits
def check_API_limits():
    global current_API_weight
    if current_API_weight >= MAX_API_WEIGHT:
        current_time = time.localtime()
        # seconds remaining until the end of the current minute
        seconds_left = 60 - current_time.tm_sec
        print(
            f"API limit exceeded with weight = {current_API_weight}. Waiting for {seconds_left} seconds.")
        time.sleep(seconds_left)
        current_API_weight = 0


# We should add start time and end time, and limit to 1000.
        
def get_klines_data(symbol, interval,startTime,endTime,limit):
    global current_API_weight
    check_API_limits()
    url = 'https://api1.binance.com'
    api_call = f'''/api/v3/klines?symbol={symbol}
                    &interval={interval}
                    &startTime={int(startTime)}
                    &endTime={int(endTime)}
                    &limit={int(limit)}'''  
    headers = {'content-type': 'application/json', 'X-MBX-APIKEY': api_key}

    response = requests.get(url + api_call, headers=headers)
    response_text = json.loads(response.text)
    current_API_weight = int(response.headers.get('x-mbx-used-weight-1m'))
    return(response_text)

In [ ]:
# Loading strategy:
# We have a max of 1000 rows by petition.
# for each day we need to retrieve 1440 rows (1 for each minute)
# So we split the day in two and do 720 as a limit and add that time to  the startin Unix Epoch

start_date = '2023-01-03 08:00:00' #(2020-01-03 08:00:00)
end_date = '2024-01-03 08:00:00' # (2020-01-05 08:00:00)

start_date_Unix = f.datetime_to_unix(start_date)
end_date_Unix = f.datetime_to_unix(end_date)
halfDay_ms = 43200000

# Define column names
columns = ['timestamp',
            'open',
            'high',
            'low', 
            'close', 
            'volume', 
            'close_time', 
            'quote_asset_volume', 
            'number_of_trades', 
            'taker_buy_base_asset_volume', 
            'taker_buy_quote_asset_volume', 
            'unused_field']

# Create an empty DataFrame
df = pd.DataFrame(columns=columns)

pointer_date = start_date_Unix
while pointer_date < end_date_Unix:
    print(f.unix_to_datetime(pointer_date))
    data = get_klines_data('BTCEUR', '1m',pointer_date,pointer_date+halfDay_ms,'720')
    df_data = pd.DataFrame(data, columns=columns)
    df = pd.concat([df,df_data])    
    pointer_date += halfDay_ms

df.to_csv('2023_BTCEUR.csv')

In [ ]:
df.to_csv('2023_BTCEUR.csv')

In [ ]:
# List of file paths
file_paths = ['2020_BTCEUR.csv', '2021_BTCEUR.csv', '2022_BTCEUR.csv','2023_BTCEUR.csv']

# Create an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Read and append each CSV file to the combined DataFrame
for file_path in file_paths:
    df = pd.read_csv(file_path)
    combined_df = pd.concat([combined_df,df])

# Export the combined DataFrame to a single CSV file
combined_df.to_csv('BTCEUR.csv', index=False)